In [ ]:
import sys

sys.path.append("../")
from copy import deepcopy

import numpy as np
import simulation_tools as sim
from cmdstanpy import CmdStanModel
from IPython.display import display

In [ ]:
# Remove cmdstanpy logs
import logging

logger = logging.getLogger("cmdstanpy")
logger.addHandler(logging.NullHandler())
logger.propagate = False
logger.setLevel(logging.CRITICAL)

In [3]:
exposed_only = sim.ExposedOnlyEstimator()
difference = sim.DifferenceEstimator()

model = CmdStanModel(stan_file="../basic_mlm.stan")
posterior_mean_fitter = sim.StanFitterBasic(model)

posterior_mean = sim.PosteriorMeanEstimator(posterior_mean_fitter)

In [ ]:
simulator1 = deepcopy(sim.CHICK_SIMULATOR)
simulator1.identifier = {"p": "p = constant 0.5"}
simulator1.mu_b = 0

simulator2 = deepcopy(sim.CHICK_SIMULATOR)
simulator2.identifier = {"p": "p = 0.5 to 0.95"}
simulator2.p = np.linspace(0.5, 0.95, sim.CHICK_J)
simulator2.mu_b = 0

In [ ]:
simulators = []
for simulator in [simulator1, simulator2]:
    for sigma_b in sim.CHICK_SIGMA_B_GRID:
        new_sim = deepcopy(simulator)
        new_sim.sigma_b = sigma_b
        new_sim.identifier = new_sim.identifier | {"sigma_b": sigma_b}
        simulators.append(new_sim)

In [6]:
aggregator = sim.SimulationAggregator(simulators)
result = aggregator.simulate(
    [exposed_only, difference, posterior_mean], 3, 0.05, show_progress=False
)

Simulators:   0%|          | 0/6 [00:00<?, ?it/s]

Estimators:   0%|          | 0/3 [00:00<?, ?it/s]

Repetition:   0%|          | 0/3 [00:00<?, ?it/s]

Repetition:   0%|          | 0/3 [00:00<?, ?it/s]

Repetition:   0%|          | 0/3 [00:00<?, ?it/s]

...

Repetition:   0%|          | 0/3 [00:00<?, ?it/s]

Estimators:   0%|          | 0/3 [00:00<?, ?it/s]

Repetition:   0%|          | 0/3 [00:00<?, ?it/s]

Repetition:   0%|          | 0/3 [00:00<?, ?it/s]

Repetition:   0%|          | 0/3 [00:00<?, ?it/s]

In [10]:
result_means = result.means()
display(result_means)

,p,sigma_b,estimator,prop_signif,sample_rmse,pop_rmse,type_s_rate,rank_corr
0,p = 0.5 to 0.95,0.0,Difference,0.350877,0.067724,0.081073,0.000000,0.556844
1,p = 0.5 to 0.95,0.0,Exposed Only,0.631579,0.032346,0.067959,0.000000,0.850093
2,p = 0.5 to 0.95,0.0,Posterior Mean,0.912281,0.037118,0.037366,0.008772,0.734471
3,p = 0.5 to 0.95,0.5,Difference,0.394737,0.079775,0.093567,0.017544,0.519203
4,p = 0.5 to 0.95,0.5,Exposed Only,0.894737,0.481388,0.479823,0.385965,0.055185
5,p = 0.5 to 0.95,0.5,Posterior Mean,0.719298,0.041530,0.024710,0.017544,0.658460
6,p = 0.5 to 0.95,1.0,Difference,0.324561,0.066191,0.086554,0.000000,0.547507
7,p = 0.5 to 0.95,1.0,Exposed Only,0.929825,1.001983,1.009412,0.438596,0.204216
8,p = 0.5 to 0.95,1.0,Posterior Mean,0.517544,0.050565,0.050827,0.008772,0.666703
9,p = constant 0.5,0.0,Difference,0.482456,0.060077,0.081928,0.017544,0.613597


In [11]:
import plotly.express as px

# define dash styles for the two p‐settings
dash_map = {"p = constant 0.5": "solid", "p = 0.5 to 0.95": "dash"}

fig_rmse = px.line(
    result_means,
    x="sigma_b",
    y="sample_rmse",
    color="estimator",
    line_dash="p",
    line_dash_map=dash_map,
    markers=True,
    title="RMSE vs. σ_b by Estimator",
    labels={"sigma_b": "σ_b", "pop_rmse": "RMSE"},
)

# keep legend showing only estimator
fig_rmse.update_layout(legend_title_text="Estimator")
fig_rmse.show()

In [12]:
fig_type_s = px.line(
    result_means,
    x="sigma_b",
    y="type_s_rate",
    color="estimator",
    line_dash="p",
    line_dash_map=dash_map,
    markers=True,
    title="Type S error rate vs. σ_b by Estimator",
    labels={"sigma_b": "σ_b", "type_s_error": "Type S Error Rate"},
)

# keep legend showing only estimator
fig_type_s.update_layout(legend_title_text="Estimator")
fig_type_s.show()

In [13]:
fig_corr = px.line(
    result_means,
    x="sigma_b",
    y="rank_corr",
    color="estimator",
    line_dash="p",
    line_dash_map=dash_map,
    markers=True,
    title="Rank Correlation vs. σ_b by Estimator",
    labels={"sigma_b": "σ_b", "rank_corr": "Rank Correlation"},
)

# keep legend showing only estimator
fig_corr.update_layout(legend_title_text="Estimator")
fig_corr.show()